In [17]:
import chardet

# Detect encoding
with open('LLCP2022.xpt', 'rb') as f:
    result = chardet.detect(f.read(10000))  # Check the first 10,000 bytes
    print(result)

{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


In [ ]:
import pandas as pd
import pyreadstat

chunk_size = 1000 
row_offset = 0
output_file = 'LLCP2022.csv'
encoding = 'windows-1252'
 
with open(output_file, 'w', encoding='utf-8') as f_out:
    while True:
        # Read a chunk of data with specified offset and limit
        df, meta = pyreadstat.read_xport('LLCP2022.xpt', encoding=encoding, row_offset=row_offset, row_limit=chunk_size)
        
        # Break the loop if no more rows are available
        if df.empty:
            break
        
        # Write header only for the first chunk
        df.to_csv(f_out, mode='a', index=False, header=(row_offset == 0), encoding='utf-8')
        
        # Update the offset for the next chunk
        row_offset += chunk_size

print("Conversion to CSV completed!")

Conversion to CSV completed!


Filtering columns to keep only the relevant ones

In [42]:
import pandas as pd

# The columns you want to keep
columns_to_keep = ['SEXVAR', 'SLEPTIM1', 'CVDINFR4', 'CVDCRHD4', 'ASTHMA3', 'CHCOCNC1', 'CHCCOPD3', 'ADDEPEV3', 'CHCKDNY2', 'DIABETE4', 'WEIGHT2',
                    'HEIGHT3', 'DEAF', 'BLIND', 'DIFFWALK', 'SMOKDAY2', 'HPVADVC4', 'COPDSMOK', 'LSATISFY', 'SDHSTRE1', 'RRCLASS3']  

In [43]:
# Load only the specified columns from the large CSV
df = pd.read_csv('LLCP2022.csv', usecols=columns_to_keep)
print("Number of rows:", len(df))

Number of rows: 445132


In [44]:
df.rename(columns={'SEXVAR':'sex', 'SLEPTIM1':'avg_sleep_time', 'CVDINFR4':'heart_attack_evr', 'CVDCRHD4':'heart_disease', 'ASTHMA3':'asthma_ever',
                    'CHCOCNC1':'any_cancer', 'CHCCOPD3':'plumonary_disease', 'ADDEPEV3':'depressive_disorder', 'CHCKDNY2':'kidney_disease', 'DIABETE4':'has_diabetes', 
                    'WEIGHT2':'weight', 'HEIGHT3':'height', 'DEAF':"is_deaf", 'BLIND':"is_blind", 'DIFFWALK':'any_walk_difficulty', 'SMOKDAY2':'smoke_habit', 
                    'HPVADVC4':'hpv_vaccin', 'COPDSMOK':'no_smoking_years', 'LSATISFY':'life_satisfaction', 'SDHSTRE1':'stress_level', 'RRCLASS3':'race'}, inplace=True)

In [45]:
print(df.isnull().sum())

sex                         0
avg_sleep_time              3
heart_attack_evr            4
heart_disease               2
asthma_ever                 2
any_cancer                  3
plumonary_disease           2
depressive_disorder         7
kidney_disease              2
has_diabetes                3
weight                  15901
height                  17055
is_deaf                 18644
is_blind                19855
any_walk_difficulty     22155
smoke_habit            281079
hpv_vaccin             437786
no_smoking_years       437917
life_satisfaction      190644
stress_level           193921
race                   283394
dtype: int64


Take each column separately:  <br>  <t> - drop rows with null values <br>
                                    <tab> - update missing values <br>
                                    <tab> - uncode some complex values <br>
                                    <tab> - drop if not relevant <br>

In [46]:
#avg_sleep_time (1 - 24	Number of hours // 77 Don’t know/Not Sure	// 99	Refused)
df["avg_sleep_time"].fillna(df["avg_sleep_time"].mean(), inplace = True)
df["avg_sleep_time"].replace(77, df["avg_sleep_time"].mean(), inplace = True)
df["avg_sleep_time"].replace(99, df["avg_sleep_time"].mean(), inplace = True)

In [ ]:
#heart_attack_evr (1 Yes   //   2 No   //   7 Don’t know/Not sure	//   9	Refused )
#will treat null, don't know and refused as a single category with value 7 -> no answer
df['heart_attack_evr'].fillna(7, inplace = True)
df['heart_attack_evr'].replace(9, 7, inplace=True)

In [51]:
#heart_disease  (1 Yes	 //   2 No	  //  7 Don’t know/Not sure	  //  9	  Refused)
#same as heart attack
df['heart_disease'].fillna(7, inplace = True)
df['heart_disease'].replace(9, 7, inplace=True)

In [ ]:
#asthma_ever (1 Yes	 //   2 No	  //  7 Don’t know/Not sure	  //  9	  Refused )
df['asthma_ever'].fillna(7, inplace = True)
df['asthma_ever'].replace(9, 7, inplace=True)

In [52]:
#any_cancer (1 Yes	 //   2 No	  //  7 Don’t know/Not sure	  //  9	  Refused )
df['any_cancer'].fillna(7, inplace = True)
df['any_cancer'].replace(9, 7, inplace=True)

In [56]:
#plumonary_disease  (1 Yes	 //   2 No	  //  7 Don’t know/Not sure	  //  9	  Refused )
df['plumonary_disease'].fillna(7, inplace = True)
df['plumonary_disease'].replace(9, 7, inplace=True)

In [57]:
#depressive_disorder  (1 Yes	 //   2 No	  //  7 Don’t know/Not sure	  //  9	  Refused )
df['depressive_disorder'].fillna(7, inplace = True)
df['depressive_disorder'].replace(9, 7, inplace=True)

In [58]:
#kidney_disease  (1 Yes	 //   2 No	  //  7 Don’t know/Not sure	  //  9	  Refused )
df['kidney_disease'].fillna(7, inplace = True)
df['kidney_disease'].replace(9, 7, inplace=True)

In [60]:
#has_diabetes (1	Yes	// 2	Yes, but female told only during pregnancy //3	No // 4	No, pre-diabetes or borderline diabetes //7	Don’t know/Not Sure //9	Refused )
#drop rows with null, 7 or 9 values
df= df.dropna(subset=['has_diabetes'])
df = df[df.has_diabetes != 7]
df = df[df.has_diabetes != 9]
# prediabetes, borderline and pregnancy dibaetes will be replaced by no so
# // it will be 1 for YES and 2 for NO
df['has_diabetes'].replace(3,2, inplace=True)
df['has_diabetes'].replace(4,2, inplace=True)

In [73]:
#weight ( 50 - 0776	Weight (pounds) 0 _ _ _ = weight in pounds	//     7777	Don’t know/Not sure	     // 
#      9023 - 9352	Weight (kilograms) The initial ´9´ indicates this was a metric value.  	//   9999	Refused)

# Convert pounds to kilograms
df.loc[(df['weight'] >= 50) & (df['weight'] <= 776), 'weight'] = df['weight'] * 0.453592

# Remove the '9' at the start for kilograms
df.loc[(df['weight'] >= 9023) & (df['weight'] <= 9352), 'weight'] = df['weight'] - 9000

# null, don't know and refused -> no answer 7777
df['weight'].fillna(7777, inplace = True)
df['weight'].replace(9999,7777, inplace=True)

In [72]:
#height ( 200 - 711	Height (ft/inches) 0 _ / _ _ = feet / inches	//     7777	Don’t know/Not sure	     // 
#      9061 - 9998	Height (meters/centimeters) The initial ´9 ´ indicates this was a metric value.	//   9999	Refused)

#pounds to cm
df.loc[(df['height'] >= 200) & (df['height'] <= 711), 'height'] =  ((df['height'] // 100) * 30.48) + ((df['height'] % 100) * 2.54)

# Remove the '9' at the start for cm
df.loc[(df['height'] >= 9061) & (df['height'] <= 9998), 'height'] = df['height'] - 9000

# null, don't know and refused -> no answer 7777
df['height'].fillna(7777, inplace = True)
df['height'].replace(9999,7777, inplace=True)

In [63]:
#deaf/hearing difficulties (1	Yes	// 2	No	// 7	Don’t know/Not Sure	// 9	Refused)

#keep  1	Yes	// 2	No	// 7	no answer
df['is_deaf'].fillna(7, inplace = True)
df['is_deaf'].replace(9, 7, inplace=True)

In [71]:
#blind/seeing difficulties (1	Yes	// 2	No	// 7	Don’t know/Not Sure	// 9	Refused)

#keep  1	Yes	// 2	No	// 7	no answer
df['is_blind'].fillna(7, inplace = True)
df['is_blind'].replace(9, 7, inplace=True)

In [70]:
#any_walk_difficulty (1	Yes	// 2	No	// 7	Don’t know/Not Sure	// 9	Refused)

#keep  1	Yes	// 2	No	// 7	no answer
df['any_walk_difficulty'].fillna(7, inplace = True)
df['any_walk_difficulty'].replace(9, 7, inplace=True)

In [74]:
print(df.isnull().sum())

sex                    0
avg_sleep_time         0
heart_attack_evr       0
heart_disease          0
asthma_ever            0
any_cancer             0
plumonary_disease      0
depressive_disorder    0
kidney_disease         0
has_diabetes           0
weight                 0
height                 0
is_deaf                0
is_blind               0
any_walk_difficulty    0
dtype: int64


In [64]:
#will drop the other columns because they have too many missing values
df.drop(['smoke_habit', 'hpv_vaccin', 'no_smoking_years','life_satisfaction','stress_level','race'], axis=1, inplace=True)

In [79]:
print("Number of rows:", len(df))

Number of rows: 444045


In [77]:
# Save to a new CSV file
filename = 'LLCP2022_filtered.csv'
import os
if os.path.exists(filename):
    os.remove(filename)
df.to_csv(filename, index=False, encoding='utf-8')

In [78]:
data = pd.read_csv('LLCP2022_filtered.csv')
print(data.isnull().sum())

sex                    0
avg_sleep_time         0
heart_attack_evr       0
heart_disease          0
asthma_ever            0
any_cancer             0
plumonary_disease      0
depressive_disorder    0
kidney_disease         0
has_diabetes           0
weight                 0
height                 0
is_deaf                0
is_blind               0
any_walk_difficulty    0
dtype: int64


In [ ]:
import pandas as pd
data = pd.read_csv('LLCP2022_filtered.csv')

61158


In [5]:
print("has diabetes: ", data["has_diabetes"].value_counts()[1])
print("dos not have dibaetes: ", data["has_diabetes"].value_counts()[2])

has diabetes:  61158
dos not have dibaetes:  382887
